# QRT ENS Challenge Data 2023 - Benchmark

Ce notebook détaille la construction du benchmark de ce challenge - il peut également être utile aux participants pour se lancer dans la compétition. 

## Librairies

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

## Chargement des données

- `X_train` et `X_test` ont  $35$ colonnes qui représentent les même variables explicatives mais sur des périodes de temps différentes. 

- `X_train` et `Y_train` partagent la même colonne `ID` - chaque ligne a un ID unique associé à un jour et à un pays. 

- La variable cible `TARGET` de `Y_train` correspond à la variation de prix journalière des futures sur l'électricité (maturité 24h).

- **On notera que certaines colonnes ont des valeurs manquantes**.


In [2]:
# After downloading the X_train/X_test/Y_train .csv files in your working directory:

X_train = pd.read_csv('X_train.csv')
Y_train = pd.read_csv('Y_train.csv')
X_test = pd.read_csv('X_test.csv')

In [5]:
X_train.head()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET
0,1054,206,FR,0.210099,-0.427458,-0.606523,0.606523,NaN,0.692860,NaN,...,-0.444661,-0.172680,-0.556356,-0.790823,-0.283160,-1.069070,-0.063404,0.339041,0.124552,-0.002445
1,2049,501,FR,-0.022399,-1.003452,-0.022063,0.022063,-0.573520,-1.130838,0.573520,...,-1.183194,-1.240300,-0.770457,1.522331,0.828412,0.437419,1.831241,-0.659091,0.047114,-0.490365
2,1924,687,FR,1.395035,1.978665,1.021305,-1.021305,-0.622021,-1.682587,0.622021,...,1.947273,-0.480700,-0.313338,0.431134,0.487608,0.684884,0.114836,0.535974,0.743338,0.204952
3,297,720,DE,-0.983324,-0.849198,-0.839586,0.839586,-0.270870,0.563230,0.270870,...,-0.976974,-1.114838,-0.507570,-0.499409,-0.236249,0.350938,-0.417514,0.911652,-0.296168,1.073948
4,1101,818,FR,0.143807,-0.617038,-0.924990,0.924990,NaN,0.990324,NaN,...,-0.526267,-0.541465,-0.424550,-1.088158,-1.011560,0.614338,0.729495,0.245109,1.526606,2.614378


In [6]:
Y_train.head()

,ID,TARGET
0,1054,0.028313
1,2049,-0.112516
2,1924,-0.180840
3,297,-0.260356
4,1101,-0.071733


## Modèles et score d'entrainement

Le benchmark pour ce challenge consiste en une simple régression linéaire, après un léger nettoyage des données : Les valeurs manquantes (NaN) ont été remplacées par des zéros et la colonne `COUNTRY` a été supprimée - en d'autres termes, nous avons utilisé un modèle identique pour la France et l'Allemagne.

In [14]:
lr = LinearRegression()

X_train_clean = X_train.drop(['COUNTRY'], axis=1).fillna(0)
Y_train_clean = Y_train['TARGET']

lr.fit(X_train_clean, Y_train_clean)

output_train = lr.predict(X_train_clean)

def metric_train(output):

    return  spearmanr(output, Y_train_clean).correlation

print("Corrélation (Spearman) pour les données d'entrainement : {:.1f}%".format(100 * metric_train(output_train) ))

Corrélation (Spearman) pour les données d'entrainement : 27.9%


La corrélation de Spearman obtenue avec notre modèle sur les données d'entrainement est d'environ 27.9%.

NB : Les variations du prix de l'électricité peuvent être très volatiles et c'est pour cette raison que nous avons préféré la corrélation de Spearman à la corrélation usuelle (Pearson), pour avoir une fonction de score plus robuste.

## Générer le benchmark

Pour générer le benchmark on procède comme on l'a fait avec les données d'entrainement mais cette fois sur les données test, et on renvoie la sortie au format csv adapté aux contraintes des sorties du challenge.  


In [15]:
X_test.head()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET
0,2136,524,FR,0.334263,-0.281509,1.042173,-1.042173,0.716838,-1.282216,-0.716838,...,-0.340008,-0.098752,-0.170879,1.020837,1.100871,0.055794,0.286185,1.485898,-0.267464,1.951327
1,900,728,DE,0.171956,-0.879758,0.912230,-0.912230,0.315475,-1.506543,-0.315475,...,-0.987289,0.789124,1.422063,0.474978,0.827341,1.544394,1.559351,-0.427837,-0.264420,-1.473669
2,375,84,DE,0.195331,-0.427774,-0.535792,0.535792,-0.927339,-0.304559,0.927339,...,-0.416412,-0.539948,-0.430405,-0.967479,-0.655789,0.160769,-0.240024,0.438932,0.163146,-0.454476
3,43,352,DE,0.393072,-0.507126,-1.378073,1.378073,-1.664931,1.349263,1.664931,...,-0.505793,5.623228,-2.046479,-0.537130,-0.781634,0.490128,-1.689460,-1.143164,-0.228964,-0.181086
4,504,24,DE,0.143362,-0.524899,-1.030414,1.030414,-1.005184,0.832404,1.005184,...,-0.635998,NaN,NaN,NaN,NaN,NaN,NaN,0.232096,-0.078845,0.190155


In [16]:
X_test_clean = X_test.drop(['COUNTRY'], axis=1).fillna(0)

Y_test_submission = X_test[['ID']].copy()
Y_test_submission['TARGET'] = lr.predict(X_test_clean)

Y_test_submission.to_csv('benchmark_qrt.csv', index=False)


On soumet alors le fichier benchmark_qrt.csv sur la site du challenge https://challengedata.ens.fr, et on obtient alors une corrélation d'environ 15.9 % - c'est le score à battre!


## Astuces et idées d'amélioration

- Réféchir à la modélation des différents facteurs qui font bouger les prix de l'électricité dans chaque pays pourra être utile. 

- Le jeu de données est relativement petit - c'est un "small data challenge" - alors attention à ne pas surapprendre les paramètres de vos modèles ! Il sera certainement utile de mettre en place de bonnes pratiques de validation croisée.
